In [1]:
import pandas as pd
from aerobot.io import load_training_data, load_validation_data
from aerobot.utls import process_data
from aerobot.models import LogisticClassifier, GeneralClassifier
import seaborn as sns

# A model based only on chemical features of the genome 

The oxidation state of DSDNA is an analytic function of its GC content. For proteins and RNA, we actually need to calculate these values, so we load a reference table. 

The code below loads amino acid and DNA sequence content for each genome, calculates the nominal oxidation states of those, and then uses them as features in a 3 class predictor of oxygen utilization. 

TODO: need to get access to coding sequences to calculate ZC of RNA coding seqs. Josh is on this. 

In [12]:
training_data = load_training_data(feature_type='chemical')
validation_data = load_validation_data(feature_type='chemical')

In [13]:
# Prepare data for classification
cleaned_data = process_data(
    training_data["features"], training_data["labels"]["physiology"],
    validation_data["features"], validation_data["labels"]["physiology"])

In [14]:
print("Fitting model...")
model = LogisticClassifier(max_iter=100000, normalize=True, random_state=None)
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])

print("Accuracy: " + str(accuracy))
print("Balanced Accuracy: " + str(balanced_accuracy))

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])
print("Test Accuracy: " + str(test_accuracy))
print("Test Balanced Accuracy: " + str(test_balanced_accuracy))

C = model.confusion_matrix(cleaned_data["X_test"], cleaned_data["y_test"])
C = pd.DataFrame(C,index=model.classifier.classes_, columns=model.classifier.classes_)
C.apply(lambda x: x/x.sum(), axis=1)


Fitting model...
Accuracy: 0.7656716417910447
Balanced Accuracy: 0.5826862710457275
Test Accuracy: 0.6183206106870229
Test Balanced Accuracy: 0.5401324029343353


,Aerobe,Anaerobe,Facultative
Aerobe,0.716049,0.271605,0.012346
Anaerobe,0.095652,0.904348,0.000000
Facultative,0.575758,0.424242,0.000000


In [15]:
# Make it a two-class thing and run again
training_data["labels"]["physiology"] = training_data["labels"]["physiology"].replace("Facultative", "Aerobe")
validation_data["labels"]["physiology"] = validation_data["labels"]["physiology"].replace("Facultative", "Aerobe")

cleaned_data = process_data(
    training_data["features"], training_data["labels"]["physiology"],
    validation_data["features"], validation_data["labels"]["physiology"])


print("Fitting two-class model...")
model = LogisticClassifier(max_iter=100000, normalize=True, random_state=None)
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])

print("Accuracy: " + str(accuracy))
print("Balanced Accuracy: " + str(balanced_accuracy))

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])
print("Test Accuracy: " + str(test_accuracy))
print("Test Balanced Accuracy: " + str(test_balanced_accuracy))

C = model.confusion_matrix(cleaned_data["X_test"], cleaned_data["y_test"])
C = pd.DataFrame(C,index=model.classifier.classes_, columns=model.classifier.classes_)
C.apply(lambda x: x/x.sum(), axis=1)

Fitting two-class model...
Accuracy: 0.8597014925373134
Balanced Accuracy: 0.8427810415291708
Test Accuracy: 0.7595419847328244
Test Balanced Accuracy: 0.7629991126885537


,Aerobe,Anaerobe
Aerobe,0.734694,0.265306
Anaerobe,0.208696,0.791304
